In [4]:
#Install required packages
!pip install --quiet pandas numpy matplotlib scipy


In [5]:
#Imports and user-config
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# show plots inline (for Jupyter)
%matplotlib inline



In [8]:
import pandas as pd

# Google Sheet ID
sheet_id = "1MTnRFZvwCDI1lnrKsQXau-zqcPzDpkg_wsnkP0wkcaA"

# Convert Google Sheet link to direct CSV export link
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

# Read directly into pandas
df = pd.read_csv(url, skiprows=3)

# Preview first few rows
print("Shape:", df.shape)
print("Columns:", list(df.columns))
display(df.head())


Shape: (98, 12)
Columns: ['Unnamed: 0', 'Date', 'unique_idfas', 'unique_ips', 'unique_uas', 'total_requests', 'requests_per_idfa', 'impressions', 'impressions_per_idfa', 'idfa_ip_ratio', 'idfa_ua_ratio', 'IVT']


,Unnamed: 0,Date,unique_idfas,unique_ips,unique_uas,total_requests,requests_per_idfa,impressions,impressions_per_idfa,idfa_ip_ratio,idfa_ua_ratio,IVT
0,NaN,11 Sep to 15 Sep,1191603,1189884,28,1770702,1.485983167,0,0,1.001444679,42557.25,0.004275535714
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Daily Data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Date,unique_idfas,unique_ips,unique_uas,total_requests,requests_per_idfa,impressions,impressions_per_idfa,idfa_ip_ratio,idfa_ua_ratio,IVT


In [9]:
#Drop completely empty rows (NaN)
df = df.dropna(how='all')

# Check sample
display(df.head(10))


,Unnamed: 0,Date,unique_idfas,unique_ips,unique_uas,total_requests,requests_per_idfa,impressions,impressions_per_idfa,idfa_ip_ratio,idfa_ua_ratio,IVT
0,NaN,11 Sep to 15 Sep,1191603,1189884,28,1770702,1.485983167,0,0,1.001444679,42557.25,0.004275535714
3,NaN,Daily Data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Date,unique_idfas,unique_ips,unique_uas,total_requests,requests_per_idfa,impressions,impressions_per_idfa,idfa_ip_ratio,idfa_ua_ratio,IVT
5,NaN,2025-09-11 0:00:00,93345,93340,22,99280,1.063581338,0,0,1.000053568,4242.954545,0.0034563
6,NaN,2025-09-12 0:00:00,515646,515307,25,649515,1.259614154,0,0,1.00065786,20625.84,0.003408541667
7,NaN,2025-09-13 0:00:00,406993,406729,25,473346,1.163032288,0,0,1.000649081,16279.72,0.006032583333
8,NaN,2025-09-14 0:00:00,400917,400668,26,467152,1.165208759,0,0,1.000621462,15419.88462,0.003835833333
9,NaN,2025-09-15 0:00:00,76526,76519,21,81409,1.063808379,0,0,1.000091481,3644.095238,0.0029675
12,NaN,Hourly Data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,Date,unique_idfas,unique_ips,unique_uas,total_requests,requests_per_idfa,impressions,impressions_per_idfa,idfa_ip_ratio,idfa_ua_ratio,IVT


In [10]:
#Convert numeric columns
num_cols = ['unique_idfas', 'unique_ips', 'unique_uas', 'total_requests',
            'requests_per_idfa', 'impressions', 'impressions_per_idfa',
            'idfa_ip_ratio', 'idfa_ua_ratio']

for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')

print(df.dtypes)


Unnamed: 0              float64
Date                     object
unique_idfas            float64
unique_ips              float64
unique_uas              float64
total_requests          float64
requests_per_idfa       float64
impressions             float64
impressions_per_idfa    float64
idfa_ip_ratio           float64
idfa_ua_ratio           float64
IVT                      object
dtype: object


In [11]:
# Clean the 'Date' column
df['Date'] = df['Date'].astype(str).str.strip()
print(df['Date'].head())


0      11 Sep to 15 Sep
3            Daily Data
4                  Date
5    2025-09-11 0:00:00
6    2025-09-12 0:00:00
Name: Date, dtype: object


In [12]:
df['requests_per_idfa'] = df['total_requests'] / df['unique_idfas']
df['impressions_per_idfa'] = df['impressions'] / df['unique_idfas']
df['idfa_ip_ratio'] = df['unique_idfas'] / df['unique_ips']
df['idfa_ua_ratio'] = df['unique_idfas'] / df['unique_uas']


In [13]:
#Compute IVT (invalid traffic) heuristic
THRESH_IDFA_UA_RATIO = 5.0
THRESH_REQUESTS_PER_IDFA = 200.0
THRESH_IMPRESSIONS_ZERO = True

df['computed_ivt'] = (
    (df['idfa_ua_ratio'] > THRESH_IDFA_UA_RATIO) |
    (df['requests_per_idfa'] > THRESH_REQUESTS_PER_IDFA) |
    ((df['impressions_per_idfa'] == 0) if THRESH_IMPRESSIONS_ZERO else False)
)

print("IVT rows count:", df['computed_ivt'].sum(), "out of", len(df))
display(df[['Date','idfa_ua_ratio','requests_per_idfa','impressions_per_idfa','computed_ivt']].head(10))


IVT rows count: 90 out of 94


,Date,idfa_ua_ratio,requests_per_idfa,impressions_per_idfa,computed_ivt
0,11 Sep to 15 Sep,42557.250000,1.485983,0.0,True
3,Daily Data,NaN,NaN,NaN,False
4,Date,NaN,NaN,NaN,False
5,2025-09-11 0:00:00,4242.954545,1.063581,0.0,True
6,2025-09-12 0:00:00,20625.840000,1.259614,0.0,True
7,2025-09-13 0:00:00,16279.720000,1.163032,0.0,True
8,2025-09-14 0:00:00,15419.884615,1.165209,0.0,True
9,2025-09-15 0:00:00,3644.095238,1.063808,0.0,True
12,Hourly Data,NaN,NaN,NaN,False
13,Date,NaN,NaN,NaN,False


In [14]:
ivt_periods = df[df['computed_ivt'] == True]['Date'].tolist()
non_ivt_periods = df[df['computed_ivt'] == False]['Date'].tolist()

print("🟥 Weeks flagged IVT:", ivt_periods)
print("🟩 Normal weeks:", non_ivt_periods)


🟥 Weeks flagged IVT: ['11 Sep to 15 Sep', '2025-09-11 0:00:00', '2025-09-12 0:00:00', '2025-09-13 0:00:00', '2025-09-14 0:00:00', '2025-09-15 0:00:00', '2025-09-11 14:00:00', '2025-09-11 15:00:00', '2025-09-11 16:00:00', '2025-09-11 17:00:00', '2025-09-11 18:00:00', '2025-09-11 19:00:00', '2025-09-11 20:00:00', '2025-09-11 21:00:00', '2025-09-11 22:00:00', '2025-09-11 23:00:00', '2025-09-12 0:00:00', '2025-09-12 1:00:00', '2025-09-12 2:00:00', '2025-09-12 3:00:00', '2025-09-12 4:00:00', '2025-09-12 5:00:00', '2025-09-12 6:00:00', '2025-09-12 7:00:00', '2025-09-12 8:00:00', '2025-09-12 9:00:00', '2025-09-12 10:00:00', '2025-09-12 11:00:00', '2025-09-12 12:00:00', '2025-09-12 13:00:00', '2025-09-12 14:00:00', '2025-09-12 15:00:00', '2025-09-12 16:00:00', '2025-09-12 17:00:00', '2025-09-12 18:00:00', '2025-09-12 19:00:00', '2025-09-12 20:00:00', '2025-09-12 21:00:00', '2025-09-12 22:00:00', '2025-09-12 23:00:00', '2025-09-13 0:00:00', '2025-09-13 1:00:00', '2025-09-13 2:00:00', '2025-09-1

In [16]:
import plotly.express as px

fig = px.line(df,
              x='Date',
              y=['requests_per_idfa','idfa_ua_ratio'],
              markers=True,
              title="Interactive Weekly Traffic Metrics Trend")

fig.update_layout(
    xaxis_tickangle=45,
    xaxis=dict(tickmode='linear', tickfont=dict(size=10)),
    legend_title_text='Metrics'
)
fig.show()


In [17]:
summary = {
    'Total Weeks': len(df),
    'IVT Weeks Count': df['computed_ivt'].sum(),
    'Normal Weeks Count': (len(df) - df['computed_ivt'].sum()),
    'IVT Weeks': ivt_periods
}
print(summary)


{'Total Weeks': 94, 'IVT Weeks Count': np.int64(90), 'Normal Weeks Count': np.int64(4), 'IVT Weeks': ['11 Sep to 15 Sep', '2025-09-11 0:00:00', '2025-09-12 0:00:00', '2025-09-13 0:00:00', '2025-09-14 0:00:00', '2025-09-15 0:00:00', '2025-09-11 14:00:00', '2025-09-11 15:00:00', '2025-09-11 16:00:00', '2025-09-11 17:00:00', '2025-09-11 18:00:00', '2025-09-11 19:00:00', '2025-09-11 20:00:00', '2025-09-11 21:00:00', '2025-09-11 22:00:00', '2025-09-11 23:00:00', '2025-09-12 0:00:00', '2025-09-12 1:00:00', '2025-09-12 2:00:00', '2025-09-12 3:00:00', '2025-09-12 4:00:00', '2025-09-12 5:00:00', '2025-09-12 6:00:00', '2025-09-12 7:00:00', '2025-09-12 8:00:00', '2025-09-12 9:00:00', '2025-09-12 10:00:00', '2025-09-12 11:00:00', '2025-09-12 12:00:00', '2025-09-12 13:00:00', '2025-09-12 14:00:00', '2025-09-12 15:00:00', '2025-09-12 16:00:00', '2025-09-12 17:00:00', '2025-09-12 18:00:00', '2025-09-12 19:00:00', '2025-09-12 20:00:00', '2025-09-12 21:00:00', '2025-09-12 22:00:00', '2025-09-12 23:00:0